## Deliverable 2. Create a Customer Travel Destinations Map

In [36]:
# Import dependencies
import pandas as pd
import requests
import gmaps

# Import API key
from config import gmaps_api_key

# Configure gmaps API key
gmaps.configure(api_key=gmaps_api_key)

In [6]:
# Read WeatherPy_database.csv file and create DF
city_weather_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_weather_df.head()

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hithadhoo,MV,-0.6000,73.0833,82.56,58,27,3.15,scattered clouds
1,1,Ambon,ID,-3.6954,128.1814,77.14,94,40,4.61,scattered clouds
2,2,Mount Isa,AU,-20.7333,139.5000,83.97,21,4,9.22,light rain
3,3,Hobart,AU,-42.8794,147.3294,53.46,84,0,6.91,clear sky
4,4,Comodoro Rivadavia,AR,-45.8667,-67.5000,73.26,40,0,12.66,clear sky


In [7]:
# Prompt user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [9]:
# Filter DF to find cities that fit temp criteria
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(5)

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hithadhoo,MV,-0.6000,73.0833,82.56,58,27,3.15,scattered clouds
1,1,Ambon,ID,-3.6954,128.1814,77.14,94,40,4.61,scattered clouds
2,2,Mount Isa,AU,-20.7333,139.5000,83.97,21,4,9.22,light rain
5,5,Hilo,US,19.7297,-155.0900,78.53,68,0,5.99,clear sky
7,7,Acajutla,SV,13.5928,-89.8275,84.09,61,48,9.55,scattered clouds


In [55]:
# Determine if there are any empty rows
preferred_cities_df.count()

City ID                225
City                   225
Country                223
Lat                    225
Lng                    225
Max Temp               225
Humidity               225
Cloudiness             225
Wind Speed             225
Current Description    225
dtype: int64

In [56]:
# Drop any empty rows and create new DF
preferred_cities_clean_df = preferred_cities_df.dropna()
preferred_cities_clean_df.count()

City ID                223
City                   223
Country                223
Lat                    223
Lng                    223
Max Temp               223
Humidity               223
Cloudiness             223
Wind Speed             223
Current Description    223
dtype: int64

In [16]:
# Create hotel DF to store hotel names along with city, country, and weather data
hotel_df = preferred_cities_clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Hithadhoo,MV,82.56,scattered clouds,-0.6000,73.0833,
1,Ambon,ID,77.14,scattered clouds,-3.6954,128.1814,
2,Mount Isa,AU,83.97,light rain,-20.7333,139.5000,
5,Hilo,US,78.53,clear sky,19.7297,-155.0900,
7,Acajutla,SV,84.09,scattered clouds,13.5928,-89.8275,


In [18]:
# Set parameters to search for hotels within 5000 meters
params = {
    "radius": 5000,
    "type": "lodging",
    "key": gmaps_api_key}

# Iterate through the hotel DF
for index, row in hotel_df.iterrows():

    # Get latitude and longitude
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add latitude and longitude to location key for params dictionary
    params["location"] = f"{lat}, {lng}"
    
    # Set up base URL
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Make request and get JSON data from search
    hotels = requests.get(base_url, params = params).json()
    
    # Get first hotel from results and store name, if a hotel isn't found skip city
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except(IndexError):
        print("Hotel not found...skipping")

Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping


In [58]:
# Drop rows with no hotel found
nan_value = float("NaN")
hotel_df.replace("", nan_value, inplace = True)
hotel_df.dropna(inplace = True)
hotel_df.count()

City                   198
Country                198
Max Temp               198
Current Description    198
Lat                    198
Lng                    198
Hotel Name             198
dtype: int64

In [59]:
# View hotel DF
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Hithadhoo,MV,82.56,scattered clouds,-0.6000,73.0833,Scoop Guest House
1,Ambon,ID,77.14,scattered clouds,-3.6954,128.1814,Swiss-Belhotel Ambon
2,Mount Isa,AU,83.97,light rain,-20.7333,139.5000,ibis Styles Mt Isa Verona
5,Hilo,US,78.53,clear sky,19.7297,-155.0900,Hilo Hawaiian Hotel
7,Acajutla,SV,84.09,scattered clouds,13.5928,-89.8275,Laureles


In [60]:
# Export hotel DF as CSV
output_data_file = "WeatherPy_Vacation.csv"
hotel_df.to_csv(output_data_file, index_label = "City ID")

In [61]:
# Use info box template to add hotel name, city name, country code, weather description, and maximum temperature for city
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# Get the data from each row to add to info box template and save in list
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# Get latitude and longitude from each row and store in new DF
locations_df = hotel_df[["Lat", "Lng"]]

In [62]:
# Create map with markers

# Assign figure variable
fig = gmaps.figure(center = (30.0, 31.0), zoom_level = 1.6)

# Assign marker variable
marker_layer = gmaps.marker_layer(locations_df, info_box_content = hotel_info)

# Add marker layer
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))